In [1]:
# Подключение библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyproj import Transformer
import xarray as xr
import datetime as dt
import os

In [2]:
# Входные данные
thunderbolts_npz_in = 'C:/Users/Maks/Desktop/Jupyter/2012_thunderbolts_clastered.npz'
thunderbolts_hdf_out = '2012_thunderbolts_clastered.h5'
satellite_directory = 'C:/Users/Maks/Desktop/Jupyter/satellite data'
output_filtered_satellite_dir = "C:/Users/Maks/Desktop/Jupyter/output_filtered_satellite_data"
interval = 15 # 15 минутный интервал вокруг грозового разряда и вокруг момента совпадения

In [3]:
# Преобразование из формата .npz в .hdf, удаление лишних данных, установление даты в качестве индекса
data = np.load(thunderbolts_npz_in)
data_npz = pd.DataFrame(data['strikes']).drop('tail', axis=1).set_index('date')
data_npz.to_hdf(thunderbolts_hdf_out, key='strikes', mode='w', complevel=9)
data_hdf = pd.read_hdf('2012_thunderbolts_clastered.h5', 'strikes')

In [4]:
# Выявление файлов .vec с некорректными данными (Data_OK=b'F')
files_to_delete = []
for filename in os.listdir(satellite_directory):
    file_path = os.path.join(satellite_directory, filename)
    satellite_data = xr.open_dataset(file_path, decode_timedelta=True)
    if (satellite_data['data_ok'].values == b'F').any():
        files_to_delete.append(file_path)
        print(f"Файл {filename} содержит data_ok=b'F'")
    satellite_data.close()
if len(files_to_delete) == 0:
    print("Файлов с data_ok=b'F' не найдено")

Файлов с data_ok=b'F' не найдено


In [5]:
# Другой метод расчета области грозового кластера
# Функция удаления отдаленных разрядов и выделения области наибольшего скопления
def simple_remove_outliers(group, percentile=90, plot=False, cluster_id=None):
    if len(group) <= 5:
        # Для малых кластеров находим самую плотную группу разрядов
        if len(group) == 1:
            # Для одного разряда - минимальная область вокруг него
            lat, lon = group['lat'].iloc[0], group['lon'].iloc[0]
            fixed_span = 0.09  # 10×10 км
            bounds = (
                lat - fixed_span/2,
                lat + fixed_span/2,
                lon - fixed_span/2,
                lon + fixed_span/2
            )
        else:
            # Для 2-5 разрядов находим пару самых близких точек
            coords = group[['lat', 'lon']].values
            min_distance = float('inf')
            closest_pair = None
            # Находим две самые близкие точки
            for i in range(len(coords)):
                for j in range(i+1, len(coords)):
                    distance = np.sqrt((coords[i][0] - coords[j][0])**2 + (coords[i][1] - coords[j][1])**2)
                    if distance < min_distance:
                        min_distance = distance
                        closest_pair = (coords[i], coords[j])
            if closest_pair:
                # Центр между двумя самыми близкими точками
                lat_center = (closest_pair[0][0] + closest_pair[1][0]) / 2
                lon_center = (closest_pair[0][1] + closest_pair[1][1]) / 2
                # Определяем размер области на основе распределения точек
                distances_from_center = np.sqrt(
                    (group['lat'] - lat_center)**2 + (group['lon'] - lon_center)**2
                )
                # Берем 75% перцентиль расстояний + минимальный размер
                radius = max(np.percentile(distances_from_center, 75), 0.045)  # минимум 5 км
                bounds = (
                    lat_center - radius,
                    lat_center + radius,
                    lon_center - radius,
                    lon_center + radius
                )
            else:
                # Запасной вариант
                lat_center = group['lat'].mean()
                lon_center = group['lon'].mean()
                fixed_span = 0.09
                bounds = (
                    lat_center - fixed_span/2,
                    lat_center + fixed_span/2,
                    lon_center - fixed_span/2,
                    lon_center + fixed_span/2
                )
    else:
        # Для больших кластеров - обычная фильтрация по перцентилям
        cut_percent = (100 - percentile) / 2
        lat_min = np.percentile(group['lat'], cut_percent)
        lat_max = np.percentile(group['lat'], 100 - cut_percent)
        lon_min = np.percentile(group['lon'], cut_percent)
        lon_max = np.percentile(group['lon'], 100 - cut_percent)
        bounds = (lat_min, lat_max, lon_min, lon_max)
    if plot:
        plot_cluster_comparison(group, bounds, cluster_id, 
                               "Dense cluster" if len(group) <= 5 else f"Filtered (percentile={percentile}%)")
    return bounds

In [6]:
# Алгоритм по поиску совпадений между пролетом спутника и областью грозового кластера
results = []
for filename in os.listdir(satellite_directory):
    file_path = os.path.join(satellite_directory, filename)
    satellite_data = xr.open_dataset(file_path, decode_timedelta=True)
    
    dataframe_satellite = pd.DataFrame({'date': satellite_data['time'], 'lat': satellite_data['lat'], 'lon': satellite_data['lon']})
    dataframe_satellite['date'] += dt.datetime(1980,1,6,0,0,0)
    dataframe_satellite = dataframe_satellite.set_index('date')
    
    # Вычисляем временной интервал для файла
    year = int(file_path[53:57])
    day_of_year = int(file_path[57:60])
    time_start = dt.datetime(year, 1, 1) + dt.timedelta(day_of_year - 1)
    time_end = time_start + dt.timedelta(1)
    
    # Фильтрация файла с данными о грозах data_hdf
    mask = (data_hdf.index >= time_start) & (data_hdf.index <= time_end)
    filtered_hdf = data_hdf[mask]
    
    if not filtered_hdf.empty:
        # Обрабатываем только кластеры с clnb > 0
        clusters = filtered_hdf[filtered_hdf['clnb'] > 0].groupby('clnb')
        
        for clnb, group in clusters:
            # Вычисляем границы, без учета отдаленных грозовых разрядов, в зоне скопления
            lat_min, lat_max, lon_min, lon_max = simple_remove_outliers(
                group, percentile=90, plot=False
            )
            
            # Проверяем каждый грозовой разряд в кластере
            for flash_time in group.index:
                # Временное окно ±15 минут вокруг грозового разряда
                time_min = flash_time - dt.timedelta(minutes=interval)
                time_max = flash_time + dt.timedelta(minutes=interval)
                
                # Поиск совпадений в спутниковых данных
                mask = (
                    (dataframe_satellite.index >= time_min) & 
                    (dataframe_satellite.index <= time_max) &
                    (dataframe_satellite['lat'] >= lat_min) & 
                    (dataframe_satellite['lat'] <= lat_max) & 
                    (dataframe_satellite['lon'] >= lon_min) & 
                    (dataframe_satellite['lon'] <= lon_max)
                )
                matched_data = dataframe_satellite[mask]
                
                # Обнаружение и обработка совпадений
                if dataframe_satellite[mask].any().any():
                    matched_data = dataframe_satellite[mask]
                    first_match = matched_data.iloc[0]
                    match_time = dataframe_satellite[mask].index[0]
                    # Находим все разряды в кластере до момента совпадения
                    flashes_before_match = group[group.index <= match_time]
                    # Последний разряд перед пролетом спутника
                    if len(flashes_before_match) > 0:
                        last_flash_before = flashes_before_match.index.max()  # Последний разряд до пролета
                        flash_times_before = flashes_before_match.index.tolist()
                    else:
                        last_flash_before = None
                        flash_times_before = []
                    results.append({
                        'clnb': clnb,
                        'satellite_file_name': filename,
                        'flash_time': last_flash_before,  # Время грозового разряда перед пролетом
                        'matched_time': match_time,  # Время пролета спутника
                        'time_dif': match_time - last_flash_before,
                        # Информация о разрядах до пролета
                        'flashes_before_count': len(flashes_before_match),
                        'flash_times_before': flash_times_before,
                        # Координаты
                        'lat_sat': first_match['lat'],   
                        'lon_sat': first_match['lon'],   
                        'lat_min': lat_min,
                        'lat_max': lat_max,
                        'lon_min': lon_min,
                        'lon_max': lon_max,
                    })
                    # Сохраняем выделенные данные пролета спутника в HDF
                    output_filename = 'trimmed_' + filename[0:-4] + '_clnb_' + str(clnb) + '.h5'
                    os.makedirs(output_filtered_satellite_dir, exist_ok=True) 
                    output_path = os.path.join(output_filtered_satellite_dir, output_filename)
                    # Фильтруем данные за ±15 минут вокруг момента совпадения
                    trim_time_min = matched_data.index[0] - dt.timedelta(minutes=interval)
                    trim_time_max = matched_data.index[0] + dt.timedelta(minutes=interval)
                    trim_mask = (
                        (dataframe_satellite.index >= trim_time_min) & 
                        (dataframe_satellite.index <= trim_time_max)
                    )
                    trimmed_data = dataframe_satellite[trim_mask]
                    # Удаление лишнего интервала пролета в файле (с одинаковым временем пролета, но другими координатами)
                    trimmed_data_reset = trimmed_data.reset_index()
                    # Находим строку совпадения по времени и координатам 
                    match_condition = (
                        (trimmed_data_reset['date'] == matched_data.index[0]) & 
                        (trimmed_data_reset['lat'] == first_match['lat']) & 
                        (trimmed_data_reset['lon'] == first_match['lon'])
                    )
                    match_idx_num = trimmed_data_reset[match_condition].index[0]
                    # Ищем разрыв ВВЕРХ (от точки совпадения к более поздним временам)
                    keep_indices_after = []
                    current_time = trimmed_data_reset.iloc[match_idx_num]['date']
                    for i in range(match_idx_num, len(trimmed_data_reset)):
                        if trimmed_data_reset.iloc[i]['date'] >= current_time:
                            keep_indices_after.append(i)
                            current_time = trimmed_data_reset.iloc[i]['date']
                        else:
                            break
                    # Ищем разрыв ВНИЗ (от точки совпадения к более ранним временам)  
                    keep_indices_before = []
                    current_time = trimmed_data_reset.iloc[match_idx_num]['date']
                    for i in range(match_idx_num, -1, -1):
                        if trimmed_data_reset.iloc[i]['date'] <= current_time:
                            keep_indices_before.append(i)
                            current_time = trimmed_data_reset.iloc[i]['date']
                        else:
                            break
                    # Объединяем индексы и берем только уникальные
                    keep_indices = list(set(keep_indices_before + keep_indices_after))
                    keep_indices.sort()
                    # Создаем отфильтрованные данные 
                    trimmed_data_filtered = trimmed_data_reset.iloc[keep_indices].set_index('date')
                    # Сохраняем в HDF
                    trimmed_data_filtered.to_hdf(output_path, key='satellite_data', mode='w')
                    break  # Прерываем после первого совпадения для этого кластера

# Создание итогового DataFrame
if results:
    satellite_overpass_matching = pd.DataFrame(results).set_index('clnb').sort_index()
else:
    satellite_overpass_matching = pd.DataFrame(columns=['satellite_file_name', 'flash_time', 'matched_time'])
    print("Совпадений не найдено")
# Вывод статистики
print(f"\nРезультаты сопоставления:")
print(f"Обработано файлов: {len(os.listdir(satellite_directory))}")
print(f"Найдено совпадений: {len(satellite_overpass_matching)}")

satellite_overpass_matching


Результаты сопоставления:
Обработано файлов: 3
Найдено совпадений: 1


,satellite_file_name,flash_time,matched_time,time_dif,flashes_before_count,flash_times_before,lat_sat,lon_sat,lat_min,lat_max,lon_min,lon_max
clnb,,,,,,,,,,,,
154,TIDI_PB_2012197_P0100_S0450_D011_R01.VEC,2012-07-15 22:40:42,2012-07-15 22:42:27,0 days 00:01:45,19,"[2012-07-15 20:45:38, 2012-07-15 20:48:22, 201...",58.045193,98.05867,57.970611,59.275,97.805056,98.727611


In [7]:
# Фаил с данными пролета спутника в момент попадания и +-15 мин
matched_data_hdf = pd.read_hdf('C:/Users/Maks/Desktop/Jupyter/output_directory\\trimmed_TIDI_PB_2012197_P0100_S0450_D011_R01_clnb_154.h5', 'satellite_data')
matched_data_hdf

,lat,lon
date,,
2012-07-15 22:29:31,31.250347,45.453651
2012-07-15 22:31:40,37.702389,49.998894
2012-07-15 22:33:50,43.707180,55.805977
2012-07-15 22:35:59,49.065868,63.280277
2012-07-15 22:38:09,53.492626,72.813087
2012-07-15 22:40:17,56.614128,84.575493
2012-07-15 22:42:27,58.045193,98.058670
2012-07-15 22:44:37,57.561176,111.948273
2012-07-15 22:46:47,55.242020,124.683838


In [8]:
flash_times = satellite_overpass_matching.loc[154, "flash_times_before"]
print(f"Все времена разрядов до попадания для кластера 154:")
for i, time in enumerate(flash_times, 1):
    print(f"{i:2d}. {time}")

Все времена разрядов до попадания для кластера 154:
 1. 2012-07-15 20:45:38
 2. 2012-07-15 20:48:22
 3. 2012-07-15 20:50:47
 4. 2012-07-15 21:06:14
 5. 2012-07-15 21:10:15
 6. 2012-07-15 21:33:05
 7. 2012-07-15 21:37:42
 8. 2012-07-15 21:44:36
 9. 2012-07-15 21:51:26
10. 2012-07-15 21:55:28
11. 2012-07-15 22:07:13
12. 2012-07-15 22:08:50
13. 2012-07-15 22:08:50
14. 2012-07-15 22:18:06
15. 2012-07-15 22:35:31
16. 2012-07-15 22:40:20
17. 2012-07-15 22:40:29
18. 2012-07-15 22:40:41
19. 2012-07-15 22:40:42


# Построение графиков результата алгоритма по удалению отдаленных разрядов и выделения области наибольшего скопления

In [9]:
# Входные данные
output_dir = "cluster_filtering_plots"
PLOT_FIRST_N = 10  # Сколько первых кластеров визуализировать
cluster_ids_to_plot=[] # Список номеров конкретных кластеров для визуализации
# Для построения графика конкретного кластера, нужно установить PLOT_FIRST_N = None

In [10]:
# Визуализация исходного кластера и результата фильтрации
def plot_cluster_comparison(original_group, bounds, cluster_id, title_suffix):
    lat_min, lat_max, lon_min, lon_max = bounds
    # Создаем маску для точек, оставшихся после фильтрации
    filtered_mask = (
        (original_group['lat'] >= lat_min) & (original_group['lat'] <= lat_max) &
        (original_group['lon'] >= lon_min) & (original_group['lon'] <= lon_max)
    )
    filtered_group = original_group[filtered_mask]
    removed_group = original_group[~filtered_mask]
    # Создаем график
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    # Левый график - исходный кластер
    ax1.scatter(original_group['lon'], original_group['lat'], c='blue', alpha=0.6, s=30, label='Все разряды')
    ax1.set_title(f'Кластер {cluster_id} - Исходные данные\n({len(original_group)} разрядов)')
    ax1.set_xlabel('Долгота')
    ax1.set_ylabel('Широта')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    # Правый график - после фильтрации
    if len(filtered_group) > 0:
        ax2.scatter(filtered_group['lon'], filtered_group['lat'], c='green', alpha=0.6, s=30, label='Оставшиеся точки')
    if len(removed_group) > 0:
        ax2.scatter(removed_group['lon'], removed_group['lat'], c='red', alpha=0.6, s=30, label='Удаленные выбросы')
    # Рисуем bounding box
    rect = plt.Rectangle((lon_min, lat_min), lon_max-lon_min, lat_max-lat_min, 
                        fill=False, edgecolor='black', linewidth=2, linestyle='--', label='Границы области')
    ax2.add_patch(rect)
    ax2.set_title(f'Кластер {cluster_id} - {title_suffix}\n({len(filtered_group)} из {len(original_group)} разрядов)')
    ax2.set_xlabel('Долгота')
    ax2.set_ylabel('Широта')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    # Настраиваем одинаковые масштабы для сравнения
    all_lons = np.concatenate([original_group['lon'], [lon_min, lon_max]])
    all_lats = np.concatenate([original_group['lat'], [lat_min, lat_max]])
    lon_padding = (all_lons.max() - all_lons.min()) * 0.1
    lat_padding = (all_lats.max() - all_lats.min()) * 0.1
    for ax in [ax1, ax2]:
        ax.set_xlim(all_lons.min() - lon_padding, all_lons.max() + lon_padding)
        ax.set_ylim(all_lats.min() - lat_padding, all_lats.max() + lat_padding)
    plt.tight_layout()
    # Сохраняем график
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(f'{output_dir}/cluster_{cluster_id}_filtering.png', dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Кластер {cluster_id}: {len(filtered_group)}/{len(original_group)} точек сохранено "
          f"({len(removed_group)} удалено)")

# Построение графиков для выбранных кластеров
def plot_selected_clusters(cluster_ids_to_plot, data_hdf, satellite_directory, PLOT_FIRST_N=None):
    plotted_count = 0
    for filename in os.listdir(satellite_directory):
        if PLOT_FIRST_N and plotted_count >= PLOT_FIRST_N:
            break    
        file_path = os.path.join(satellite_directory, filename)
        satellite_data = xr.open_dataset(file_path, decode_timedelta=True)
        dataframe_satellite = pd.DataFrame({'date': satellite_data['time'], 'lat': satellite_data['lat'], 'lon': satellite_data['lon']})
        dataframe_satellite['date'] += dt.datetime(1980,1,6,0,0,0)
        dataframe_satellite = dataframe_satellite.set_index('date')
        # Вычисляем временной интервал для файла
        year = int(file_path[53:57])
        day_of_year = int(file_path[57:60])
        time_start = dt.datetime(year, 1, 1) + dt.timedelta(day_of_year - 1)
        time_end = time_start + dt.timedelta(1)
        # Фильтрация data_hdf за один проход
        mask = (data_hdf.index >= time_start) & (data_hdf.index <= time_end)
        filtered_hdf = data_hdf[mask]
        if not filtered_hdf.empty:
            clusters = filtered_hdf[filtered_hdf['clnb'] > 0].groupby('clnb')
            for clnb, group in clusters:
                # Проверяем, нужно ли строить график для этого кластера
                should_plot = False
                if PLOT_FIRST_N and plotted_count < PLOT_FIRST_N:
                    should_plot = True
                elif cluster_ids_to_plot and clnb in cluster_ids_to_plot:
                    should_plot = True
                if should_plot:
                    print(f"Построение графика для кластера {clnb}...")
                    lat_min, lat_max, lon_min, lon_max = simple_remove_outliers(
                        group, percentile=90, plot=True, cluster_id=clnb
                    )
                    plotted_count += 1
                    if PLOT_FIRST_N and plotted_count >= PLOT_FIRST_N:
                        break
        if PLOT_FIRST_N and plotted_count >= PLOT_FIRST_N:
            break
    print(f"Построено графиков: {plotted_count}")

# Построение графиков
plot_selected_clusters(cluster_ids_to_plot=cluster_ids_to_plot, data_hdf=data_hdf, 
                      satellite_directory=satellite_directory, PLOT_FIRST_N=PLOT_FIRST_N)

Построение графика для кластера 2...
Кластер 2: 17/24 точек сохранено (7 удалено)
Построение графика для кластера 3...
Кластер 3: 727/875 точек сохранено (148 удалено)
Построение графика для кластера 4...
Кластер 4: 232/280 точек сохранено (48 удалено)
Построение графика для кластера 6...
Кластер 6: 4/6 точек сохранено (2 удалено)
Построение графика для кластера 7...
Кластер 7: 614/717 точек сохранено (103 удалено)
Построение графика для кластера 8...
Кластер 8: 16/22 точек сохранено (6 удалено)
Построение графика для кластера 9...
Кластер 9: 20/26 точек сохранено (6 удалено)
Построение графика для кластера 10...
Кластер 10: 2/6 точек сохранено (4 удалено)
Построение графика для кластера 11...
Кластер 11: 5/7 точек сохранено (2 удалено)
Построение графика для кластера 12...
Кластер 12: 14/17 точек сохранено (3 удалено)
Построено графиков: 10
